# Auditing Office Macros

### Requirements

In [ ]:
!pip install evtx

## Parsing EVTX files to Pandas DataFrames

In [1]:
from evtx import PyEvtxParser
import json
import pandas as pd
import os

In [2]:
def evtx_folder_to_dataframes(directory):
    dataframes_list_one_five = {}
    dataframes_list_seven = {}

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            events_one_five = []
            events_seven = []

            a = open(f, 'rb')
            parser = PyEvtxParser(a)

            for record in parser.records_json():
                event = json.loads(record['data'])
                #Process creation
                if event["Event"]["System"]["EventID"] == 1:
                    event_list = [event["Event"]["EventData"]["UtcTime"],
                    event["Event"]["System"]["EventID"],
                    event["Event"]["EventData"]["ProcessGuid"],
                    event["Event"]["EventData"]["ProcessId"],
                    event["Event"]["EventData"]["Image"],
                    event["Event"]["EventData"]["IntegrityLevel"],
                    event["Event"]["EventData"]["TerminalSessionId"],
                    event["Event"]["EventData"]["User"]]
                    events_one_five.append(event_list)
                if event["Event"]["System"]["EventID"] == 5:
                    event_list = [event["Event"]["EventData"]["UtcTime"],
                    event["Event"]["System"]["EventID"],
                    event["Event"]["EventData"]["ProcessGuid"],
                    event["Event"]["EventData"]["ProcessId"],
                    event["Event"]["EventData"]["Image"],[],[],[]]
                    events_one_five.append(event_list)
                #Image loaded
                if event["Event"]["System"]["EventID"] == 7:
                    event_list = [event["Event"]["EventData"]["UtcTime"],
                    event["Event"]["System"]["EventID"],
                    event["Event"]["EventData"]["ImageLoaded"],
                    event["Event"]["EventData"]["FileVersion"],
                    event["Event"]["EventData"]["Description"],
                    event["Event"]["EventData"]["Product"],
                    event["Event"]["EventData"]["Company"],
                    event["Event"]["EventData"]["OriginalFileName"],
                    event["Event"]["EventData"]["Hashes"],
                    event["Event"]["EventData"]["Signed"],
                    event["Event"]["EventData"]["Signature"],
                    event["Event"]["EventData"]["SignatureStatus"]]
                    events_seven.append(event_list)
            name = filename.split("\\")[-1].split(".")[-2]

            df_1_5 = pd.DataFrame.from_records(events_one_five,
                                   columns=['UtcTime','EventID', 'ProcessGuid', 'ProcessId', 'Image', 
                                            'IntegrityLevel', 'TerminalSessionId', 'User'])
            df_7 = pd.DataFrame.from_records(events_seven,
                                   columns=['UtcTime','EventID', 'ImageLoaded', 'FileVersion', 'Description',
                                            'Product', 'Company', 'OriginalFileName', 'Hashes', 'Signed',
                                           'Signature', 'SignatureStatus'])
            dataframes_list_one_five[name] = df_1_5
            dataframes_list_seven[name] = df_7
            
    return dataframes_list_one_five, dataframes_list_seven

In [3]:
directory = r"E:\Path\Macros\evtx"
dataframes_list_one_five, dataframes_list_seven = evtx_folder_to_dataframes(directory)

## How many libraries loads Office in each execution?

In [28]:
df_count = pd.DataFrame(index=list(dataframes_list_seven.keys()), columns=["Count"])

In [34]:
for e in list(dataframes_list_seven):
    df_count["Count"][e] = dataframes_list_seven[e]["ImageLoaded"].nunique()

In [35]:
df_count

,Count
without_2003_down,198
without_2003_local,172
without_xlsm_down,195
without_xlsm_local,174
without_xlsx_down,195
without_xlsx_local,172
with_2003_down,200
with_2003_local,178
with_4_0_xls_local,175
with_4_xls_local,173


## Differences between runs
How many libraries are loaded in one execution that are not in the other?

In [6]:
def compare_df(df_in, df_not_in):
    list = df_in.merge(df_not_in.drop_duplicates(), 
                       on=['ImageLoaded'], 
                       how='left', 
                       indicator=True) 
    list_min = list[list['_merge'] == 'left_only']["ImageLoaded"].unique()
    return(list_min)

In [7]:
df = pd.DataFrame(index=list(dataframes_list_seven.keys()), columns=list(dataframes_list_seven.keys()))

In [8]:
for e in list(dataframes_list_seven.keys()):
    for i in list(dataframes_list_seven.keys()):
        list_min = compare_df(dataframes_list_seven[e], dataframes_list_seven[i])
        df[i][e] = len(list_min)

In [9]:
df

,without_2003_down,without_2003_local,without_xlsm_down,without_xlsm_local,without_xlsx_down,without_xlsx_local,with_2003_down,with_2003_local,with_4_0_xls_down,with_4_0_xls_local
without_2003_down,0,26,3,24,3,26,12,20,24,23
without_2003_local,0,0,1,1,1,1,9,1,11,0
without_xlsm_down,0,24,0,22,0,23,11,18,21,23
without_xlsm_local,0,3,1,0,1,2,11,1,12,1
without_xlsx_down,0,24,0,22,0,23,11,18,21,23
without_xlsx_local,0,1,0,0,0,0,10,1,11,1
with_2003_down,1,24,3,24,3,25,0,19,13,22
with_2003_local,0,7,1,5,1,7,10,0,11,5
with_4_0_xls_down,0,13,0,12,0,13,0,7,0,12
with_4_0_xls_local,0,3,3,2,3,4,10,2,13,0


## Comparative menu

In [17]:
def compare_events():
    import ipywidgets as widgets
    from IPython.display import display, Markdown, clear_output
    pd.set_option('display.max_rows', 500)
    output = widgets.Output()
    dfs = list(dataframes_list_seven.keys())
    columns = dataframes_list_seven[list(dataframes_list_seven.keys())[0]].columns
    in_widget = widgets.Dropdown(
                        options=dfs,
                        description='Events in:',
                        disabled=False)
    not_in_widget = widgets.Dropdown(
                        options=dfs,
                        description='And not in:',
                        disabled=False)
    columns = widgets.Dropdown(
                        options=columns,
                        description='Column:',
                        disabled=False)
    button = widgets.Button(description=f'List')
    display(in_widget, not_in_widget, columns, button, output)

    def _click_function(_):
        with output:
            clear_output()
            list = dataframes_list_seven[in_widget.value].merge(dataframes_list_seven[not_in_widget.value].drop_duplicates(), 
                                                          on=['ImageLoaded'], 
                                                          how='left', 
                                                          indicator=True) 
            list_min = list[list['_merge'] == 'left_only'][columns.value].unique()
            display(len(list_min))
            display(pd.DataFrame(list_min).style.set_properties(**{'text-align': 'left'}))
            

    button.on_click(_click_function)

In [18]:
compare_events()

Dropdown(description='Events in:', options=('without_2003_down', 'without_2003_local', 'without_xlsm_down', 'w…

Dropdown(description='And not in:', options=('without_2003_down', 'without_2003_local', 'without_xlsm_down', '…

Dropdown(description='Column:', options=('UtcTime', 'EventID', 'ImageLoaded', 'FileVersion', 'Description', 'P…

Button(description='List', style=ButtonStyle())

Output()